In [3]:
using JuMP
using HiGHS
using CSV
using DataFrames

In [4]:
data = CSV.read("gerrymander.csv", DataFrame)

Row,county_number,county_name,current_district,vote_diff_d_minus_r,vote_diff_d_minus_r_scenario_2,vote_diff_d_minus_r_scenario_3
,Int64,String15,Int64,Int64,Int64,Int64
1,1,Bernalillo,1,42941,43411,11336
2,2,Catron,2,-917,18,-716
3,3,Chaves,2,-6650,-6244,-6436
4,4,Cibola,2,1941,1449,1025
5,5,Colfax,3,116,-871,-1099
6,6,Curry,3,-5194,-4241,-5093
7,7,DeBaca,2,-299,223,567
8,8,Dona Ana,2,9790,8856,8251
9,9,Eddy,2,-6436,-6787,-6736


In [5]:
counties = data.county_number
districts = 1:3
vote_diff = Dict(row.county_number => row.vote_diff_d_minus_r for row in eachrow(data))
current_districts = data.current_district

33-element Vector{Int64}:
 1
 2
 2
 2
 3
 3
 2
 2
 2
 2
 2
 3
 2
 ⋮
 3
 2
 3
 3
 3
 3
 2
 2
 3
 1
 3
 2

In [6]:
model = Model(HiGHS.Optimizer)
@variable(model, x[districts, counties], Bin)
@objective(model, Max, sum((current_districts[j] == i) * x[i, j] for i in districts, j in counties))
for j in counties
    @constraint(model, sum(x[i, j] for i in districts) == 1)
end
@constraint(model, sum(vote_diff[j] * x[1, j] for j in counties) >= 100)  # District 1
@constraint(model, sum(vote_diff[j] * x[2, j] for j in counties) >= 100)  # District 2
@constraint(model, sum(vote_diff[j] * x[3, j] for j in counties) >= 100)  # District 3

42941 x[3,1] - 917 x[3,2] - 6650 x[3,3] + 1941 x[3,4] + 116 x[3,5] - 5194 x[3,6] - 299 x[3,7] + 9790 x[3,8] - 6436 x[3,9] + 1723 x[3,10] + 870 x[3,11] - 66 x[3,12] + 99 x[3,13] - 8412 x[3,14] - 3009 x[3,15] + 395 x[3,16] - 81 x[3,17] + 9943 x[3,18] + 1361 x[3,19] - 5504 x[3,20] - 812 x[3,21] + 8016 x[3,22] - 2313 x[3,23] + 2707 x[3,24] - 13091 x[3,25] + 6473 x[3,26] + 34523 x[3,27] - 965 x[3,28] + 1285 x[3,29] + 9145 x[3,30] - 1107 x[3,31] - 760 x[3,32] + 685 x[3,33] ≥ 100

In [9]:
optimize!(model)
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found!")
    println("Democratic margin in District 2: ", objective_value(model))

    # Output the county assignments for each district
    for i in districts
        assigned_counties = [counties[j] for j in counties if value(x[i, j]) > 0.5]
        println("District $i: Counties $assigned_counties")
    end
    
    # Count how many counties are not given new assignments
    unchanged_counties = 0
    for j in counties
        if current_districts[j] == findfirst(x[:, j] > 0.5)  # Check if county j is in the same district
            unchanged_counties += 1
        end
    end
    println("Number of counties not given new assignments: $unchanged_counties")
else
    println("No optimal solution found.")
end

Coefficient ranges:
  Matrix [1e+00, 4e+04]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+02]
Presolving model
36 rows, 99 cols, 198 nonzeros  0s
36 rows, 99 cols, 198 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   36 rows
   99 cols (99 binary, 0 integer, 0 implied int., 0 continuous)
   198 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   33              -inf                 inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   32.57399688     31                 5.08%        0      0      0        13     0.0s
 T       0       0         0   0.00%   32.57399688     32                 1.79%        0      0      0        14     0.0s
         1       0       

LoadError: MethodError: no method matching isless(::Float64, ::JuMP.Containers.DenseAxisArray{VariableRef, 1, Tuple{UnitRange{Int64}}, Tuple{JuMP.Containers._AxisLookup{Tuple{Int64, Int64}}}})
The function `isless` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  isless([91m::Missing[39m, ::Any)
[0m[90m   @[39m [90mBase[39m [90m[4mmissing.jl:87[24m[39m
[0m  isless(::Any, [91m::Missing[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mmissing.jl:88[24m[39m
[0m  isless(::AbstractFloat, [91m::ForwardDiff.Dual{Ty}[39m) where Ty
[0m[90m   @[39m [36mForwardDiff[39m [90m~/.julia/packages/ForwardDiff/UBbGT/src/[39m[90m[4mdual.jl:149[24m[39m
[0m  ...


In [15]:
using JuMP
using HiGHS
using CSV
using DataFrames

file_path = "gerrymander.csv"
data = CSV.File(file_path)
df = DataFrame(data)
county_numbers = df[!, :county_number]
county_names = df[!, :county_name]
current_districts = df[!, :current_district]
vote_diff_d_minus_r = df[!, :vote_diff_d_minus_r]
vote_diff_d_minus_r_scenario_2 = df[!, :vote_diff_d_minus_r_scenario_2]
vote_diff_d_minus_r_scenario_3 = df[!, :vote_diff_d_minus_r_scenario_3]
num_districts = 3
num_counties = length(county_numbers)

model = Model(HiGHS.Optimizer)
@variable(model, x[1:num_districts, 1:num_counties], Bin)
@objective(model, Max, sum(x[i, j] * (current_districts[j] == i) for i in 1:num_districts, j in 1:num_counties))
for j in 1:num_counties
    @constraint(model, sum(x[i, j] for i in 1:num_districts) == 1)
end
@constraint(model, sum(vote_diff_d_minus_r[j] * x[2, j] for j in 1:num_counties) >= 100)

optimize!(model)
assignments = Dict()
unchanged_counties = 0
for j in 1:num_counties
    assigned_district = findfirst(i -> value(x[i, j]) == 1, 1:num_districts)
    assignments[county_names[j]] = assigned_district
    if current_districts[j] == assigned_district
        unchanged_counties += 1
    end
end

println("Number of counties not given new assignments: $unchanged_counties")
for county in county_names
    println("County: $county, Assigned District: ", assignments[county])
end


Running HiGHS 1.8.0 (git hash: fcfb53414): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 4e+04]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+02]
Presolving model
34 rows, 99 cols, 132 nonzeros  0s
1 rows, 33 cols, 33 nonzeros  0s
1 rows, 19 cols, 19 nonzeros  0s
1 rows, 19 cols, 19 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   1 rows
   19 cols (18 binary, 1 integer, 0 implied int., 0 continuous)
   19 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   33              -inf                 inf        0      0      0         0     0.0s
         1       0         1 100.00%   32              32                 0.00%        0      0      0         1   